In [18]:
from Data_Module import get_df_feats
df_feats, feat_cols = get_df_feats(refresh=False, persist=True)

[WARN] Could not save parquet; saving CSV instead.
 Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.


In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing

In [17]:
df_feats

,player_id,season_name,goals,assists,penalty_goals,own_goals,yellow_cards,second_yellow_cards,direct_red_cards,minutes_played,...,minutes_played_z_delta,matches_played_z_lag1,matches_played_z_delta,ga_per90_w,g_per90_w,a_per90_w,gc_per90_w,clean_sheet_rate_w,discipline_rate_w,pen_share_w
0,3333,02/03,2.0,0,0,0,0,0,0,255.0,...,0.000000,0.000000,0.000000,-0.481590,0.246093,-0.727683,0.0,0.0,-0.126468,-0.148413
1,3333,03/04,3.0,1,0,0,2,0,0,793.0,...,0.000000,0.000000,0.000000,-0.733500,-0.119311,-0.614190,0.0,0.0,-0.059801,-0.148413
2,3333,04/05,1.0,1,0,0,0,0,0,1264.0,...,0.000000,0.000000,0.000000,-1.045068,-0.388587,-0.656480,0.0,0.0,-0.126468,-0.148413
3,3333,05/06,1.0,1,0,0,3,0,0,2337.0,...,0.000000,0.000000,0.000000,-1.110451,-0.421279,-0.689172,0.0,0.0,-0.026468,-0.148413
4,3333,06/07,3.0,7,0,0,3,0,0,894.0,...,1.149865,0.000000,1.154701,-0.180761,-0.157776,-0.022985,0.0,0.0,-0.040753,-0.148413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12198,1296876,24/25,0.0,0,0,0,0,0,0,0.0,...,-0.742170,0.000000,-1.230851,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
12199,1296876,25/26,0.0,0,0,0,0,0,0,0.0,...,0.357290,-1.230851,1.230851,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
12200,1314671,24/25,0.0,0,0,0,0,0,0,0.0,...,-0.711133,0.000000,-1.687529,0.000000,0.000000,0.000000,0.0,0.0,-0.500000,0.000000
12201,1314671,25/26,0.0,0,0,0,1,0,0,0.0,...,0.405594,-1.687529,1.687529,0.000000,0.000000,0.000000,0.0,0.0,0.500000,0.000000


correlation analysis 

In [ ]:
safe_cols = [
    "goals","assists","penalty_goals","own_goals","yellow_cards","second_yellow_cards",
    "direct_red_cards","minutes_played","goals_conceded","clean_sheets","nb_on_pitch",
    "nb_in_group","subed_in","subed_out","height","matches_played","g_per90","a_per90",
    "ga_per90","pen_share","red_cards_total","discipline_rate","gc_per90","clean_sheet_rate",
]

X = df_feats[safe_cols].replace([np.inf, -np.inf], np.nan).dropna()
corr = X.corr(method="pearson")

# wrap labels to multiple lines (split underscores + wrap to width)
def wrap_label(s, width=12):
    s = s.replace("_", " ")
    return "\n".join(textwrap.wrap(s, width=width))

xlabels = [wrap_label(c, width=12) for c in corr.columns]
ylabels = [c.replace("_", " ") for c in corr.index]

fig, ax = plt.subplots(figsize=(12, 10))
im = ax.imshow(corr.values, interpolation="nearest", aspect="equal")
fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
ax.set_title("Correlation Heatmap (Pearson)")

n = len(corr.columns)
ticks = np.arange(n)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(xlabels, rotation=45, ha="right", va="top")
ax.set_yticklabels(ylabels)

# add light gridlines between cells
ax.set_xticks(np.arange(-0.5, n, 1), minor=True)
ax.set_yticks(np.arange(-0.5, n, 1), minor=True)
ax.grid(which="minor", linewidth=0.5)
ax.tick_params(axis="x", labelsize=9, pad=6)
ax.tick_params(axis="y", labelsize=9)

# give bottom extra space for the wrapped labels
plt.subplots_adjust(bottom=0.25)
plt.tight_layout()
plt.show()


In [ ]:
from statsmodels.tsa.stattools import adfuller

# Run ADF test on the selected column
series = df_feats["season_end_year"].dropna()  # drop missing values just in case

result = adfuller(series)

# Display results
print(f"ADF Statistic: {result[0]:.4f}")
print(f"p-value: {result[1]:.4f}")

# Optional: add interpretation
if result[1] < 0.05:
    print("✅ The 'season_end_year' series is stationary.")
else:
    print("⚠️ The 'season_end_year' series is non-stationary — consider differencing or detrending.")

In [ ]:
from statsmodels.tsa.stattools import adfuller
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1) Elige las métricas a evaluar por jugador en el tiempo
metrics = [
    # Z-scores
    "ga_per90_z", "g_per90_z", "a_per90_z", "gc_per90_z",
    "clean_sheet_rate_z", "discipline_rate_z", "pen_share_z",
    "minutes_played_z", "matches_played_z",
    "goals_z", "assists_z", "penalty_goals_z", "own_goals_z",
    "yellow_cards_z", "second_yellow_cards_z", "direct_red_cards_z",

    # Lags (t-1) de Z-scores
    "ga_per90_z_lag1", "g_per90_z_lag1", "a_per90_z_lag1",
    "gc_per90_z_lag1", "clean_sheet_rate_z_lag1", "discipline_rate_z_lag1",
    "pen_share_z_lag1", "minutes_played_z_lag1", "matches_played_z_lag1",

    # Deltas (cambios) de Z-scores
    "ga_per90_z_delta", "g_per90_z_delta", "a_per90_z_delta",
    "gc_per90_z_delta", "clean_sheet_rate_z_delta", "discipline_rate_z_delta",
    "pen_share_z_delta", "minutes_played_z_delta", "matches_played_z_delta",

    # Versiones weighted / winsorizadas
    "ga_per90_w", "g_per90_w", "a_per90_w", "gc_per90_w",
    "clean_sheet_rate_w", "discipline_rate_w", "pen_share_w",
]
# 2) Calcula p-values ADF por jugador para cada métrica
def adf_pvalues_by_metric(df, metrics, min_n=5):
    out = {}
    gdf = df.sort_values("season_end_year").groupby("player_id")
    for col in metrics:
        pvals = []
        for pid, g in gdf:
            s = g[col].dropna()
            if len(s) >= min_n and s.nunique() > 1:
                try:
                    pvals.append(adfuller(s)[1])  # p-value
                except Exception:
                    pass
        if pvals:
            out[col] = np.array(pvals)
    return out

pvals_dict = adf_pvalues_by_metric(df_feats, metrics, min_n=5)

# 3) Construye tabla resumen y ordénala
rows = []
for col, arr in pvals_dict.items():
    share_stat = (arr < 0.05).mean()
    rows.append({"metric": col, "n_players_tested": len(arr), "share_stationary": share_stat})

summary = pd.DataFrame(rows).sort_values("share_stationary", ascending=False).reset_index(drop=True)
summary


In [ ]:
def metric_type(name: str) -> str:
    if "_delta" in name: return "delta"
    if "_lag1" in name:  return "lag1"
    if name.endswith("_w"): return "w"
    return "z"

summary["type"] = summary["metric"].apply(metric_type)

# One figure per type (keeps each plot readable)
for t in ["delta", "lag1", "z", "w"]:
    sub = summary[summary["type"] == t]
    if sub.empty: 
        continue
    plt.figure(figsize=(10, 4))
    plt.bar(sub["metric"], sub["share_stationary"])
    plt.xticks(rotation=60, ha="right")
    plt.ylim(0, 1.0)
    plt.xlabel(f"metric ({t})")
    plt.ylabel("share stationary (ADF p<0.05)")
    plt.title(f"Stationarity share – {t} features")
    plt.tight_layout()
    plt.show()


In [ ]:
summary_display = summary.copy()
summary_display["share_stationary"] = summary_display["share_stationary"].round(3)
summary_display


In [ ]:
df_num = df_feats.select_dtypes(include=[np.number]).drop(
    columns=['player_id', 'team_id'], errors='ignore'
)
# make sure everything is float so pandas can build the covariance matrix
df_num = df_num.astype(float)
# compute covariance matrix
M = df_num.cov()
# eigenvalues and eigenvectors
from numpy import linalg as LA
w, v = LA.eig(M)
w  # eigenvalues

In [ ]:
pca = PCA()
pca.fit(df_num)


In [ ]:
pca.components_

In [ ]:
pca.explained_variance_

In [ ]:
pp = pca.transform(df_num)
def biplot(score,coeff,pcax,pcay,labels=None, Labels=None):
    pca1=pcax-1
    pca2=pcay-1
    xs = score[:,pca1]
    ys = score[:,pca2]
    n=score.shape[1]
    scalex = 1/(xs.max()- xs.min())
    scaley = 1/(ys.max()- ys.min())
    plt.scatter(xs*scalex,ys*scaley)
    for i in range(len(xs)): # Iterate through the number of samples
        if Labels is None:
            pass # Placeholder for empty if block
        else:
            plt.text(xs[i]*scalex, ys[i]*scaley, Labels[i], color='blue', ha='center', va='center')
    for i in range(n):
        plt.arrow(0, 0, coeff[i,pca1], coeff[i,pca2],color='r',alpha=0.5)
        if labels is None:
            plt.text(coeff[i,pca1]* 1.15, coeff[i,pca2] * 1.15, "Var"+str(i+1), color='g', ha='center', va='center')
        else:
            plt.text(coeff[i,pca1]* 1.15, coeff[i,pca2] * 1.15, labels[i], color='g', ha='center', va='center')
    plt.xlim(-1,1)
    plt.ylim(-1,1)
    plt.xlabel("PC{}".format(pcax))
    plt.ylabel("PC{}".format(pcay))
    plt.grid()

In [ ]:
biplot(pp,pca.components_,1,2)
plt.show()

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X1 = min_max_scaler.fit_transform(df_num)

In [ ]:
pca = PCA()
X_pca=pca.fit(X1)

In [ ]:
pca.components_

In [ ]:
pca.explained_variance_

In [ ]:
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

# --- Visualize Explained variance ---
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(explained_variance)+1), cumulative_variance, marker='o', linestyle='--', color='b')
plt.title('Cumulative Explained Variance by PC')
plt.xlabel('Number of PCs')
plt.ylabel('Cumulative Variance')
plt.grid(True)
plt.show()


In [ ]:
var_exp_df = pd.DataFrame({
    'Component': [f'PC{i+1}' for i in range(len(explained_variance))],
    'Explained Variance': explained_variance,
    'Cumulative Variance': cumulative_variance
})
print(var_exp_df)

In [ ]:
features = df_num # solo las variables
labels = df_feats["player_id"]  # abreviaturas de los estados

# Escalar los datos
X_scaled = StandardScaler().fit_transform(features)

# PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Obtener vectores de las variables originales
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

In [ ]:
# Crear el biplot
plt.figure(figsize=(10, 7))

# 1. Puntos (proyecciones de los estados)
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.7)

# 2. Etiquetas (abreviaturas de los estados)
for i, txt in enumerate(labels):
    plt.text(X_pca[i, 0], X_pca[i, 1], txt, fontsize=9, color='blue', ha='center', va='center')

# 3. Flechas de las variables originales
for i, var in enumerate(features.columns):
    plt.arrow(0, 0, loadings[i, 0], loadings[i, 1], color='brown', alpha=0.7, head_width=0.02)
    plt.text(loadings[i, 0]*1.15, loadings[i, 1]*1.15, var, color='green', ha='center', va='center', fontsize=10)

# 4. Ejes y título
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% var)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% var)")
plt.title("PCA Biplot between leagues")
plt.grid(True)
plt.axhline(0, color='grey', lw=1)
plt.axvline(0, color='grey', lw=1)
plt.tight_layout()
plt.show()


 Linear model selection and regularization: LASSO Regression, a shrinkage method


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [ ]:


X_train, X_test, y_train, y_test = train_test_split(df_num, df_num, test_size=0.3, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Fit Lasso regression model
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

In [ ]:
# Evaluate model performance on test set
y_pred = lasso.predict(X_test)
print(y_pred.mean()) #Mean Square Error over the test set

In [ ]:
# Model Score
print("Model Score: ", lasso.score(X_test, y_test))

In [ ]:
# Lasso Coefficient
lasso.coef_

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10]}

np.random.seed(30) #we fix the seed so we can reproduce the data any time

# Perform grid search with cross-validation
lasso_cv = GridSearchCV(Lasso(), param_grid, cv=5)
lasso_cv.fit(X_train, y_train)

# Print best parameter values and score
print("Best Parameters:", lasso_cv.best_params_)
print("Best Score:", lasso_cv.best_score_)

In [ ]:
#Let's extract the best value of alpha
lasso_cv.best_params_.get('alpha')

In [ ]:
#And re-evaluate the Lasso regression model with this alpha
lasso = Lasso(alpha=lasso_cv.best_params_.get('alpha'))
lasso.fit(X_train, y_train)
# Evaluate model performance on test set
y_pred = lasso.predict(X_test)
print(y_pred.mean()) #Mean Square Error over the test set

In [ ]:
# Model Score
print("Model Score: ", lasso.score(X_test, y_test))

In [ ]:
# Lasso Coefficient
lasso.coef_

factor analysis


In [ ]:
import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler

# Keep only numeric cols and drop IDs
X = (df_feats.select_dtypes(include=[np.number])
         .drop(columns=['player_id','team_id'], errors='ignore')
         .replace([np.inf, -np.inf], np.nan)
         .dropna(axis=0))

# Drop constant columns (variance = 0)
X = X.loc[:, X.nunique(dropna=True) > 1]

# Standardize (recommended)
Xz = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns, index=X.index)


In [ ]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo

chi2, p = calculate_bartlett_sphericity(Xz)   # want p < 0.05
kmo_per_var, kmo_model = calculate_kmo(Xz)    # >0.6 OK, >0.7 good, >0.8 great
print(f"Bartlett p={p:.3e}, KMO={kmo_model:.3f}")
